In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:1]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fruit-and-vegetable-image-recognition/validation/capsicum/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/sweetcorn/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/orange/Image_1.png
/kaggle/input/fruit-and-vegetable-image-recognition/validation/tomato/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/turnip/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/ginger/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/raddish/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/pomegranate/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/pineapple/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/jalepeno/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/apple/Image_4.jpg
/kaggle/input/fruit-and-vegetable-image-recognition/validation/carrot/Image_2.jpg
/

In [5]:
from __future__ import print_function

import argparse
import json
import os
import random
import time

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
from torch.optim.lr_scheduler import StepLR


# ────────────────────────────────────────────────────────────────
# Utils
# ────────────────────────────────────────────────────────────────
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def count_params(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


@torch.no_grad()
def accuracy_top1(logits, targets) -> float:
    preds = logits.argmax(dim=1)
    return (preds == targets).float().mean().item()


def ensure_dir(path: str):
    if path and not os.path.exists(path):
        os.makedirs(path, exist_ok=True)


# ────────────────────────────────────────────────────────────────
# Train / Eval
# ────────────────────────────────────────────────────────────────
def train_one_epoch(args, model, device, train_loader, optimizer, epoch):
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    seen = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        output = model(data)
        loss = nn.functional.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        bs = data.size(0)
        seen += bs
        running_loss += loss.item() * bs
        running_acc += accuracy_top1(output, target) * bs

        if batch_idx % args.log_interval == 0:
            pct = 100.0 * batch_idx / len(train_loader)
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAcc: {:.2f}%".format(
                    epoch,
                    batch_idx * bs,
                    len(train_loader.dataset),
                    pct,
                    loss.item(),
                    100.0 * accuracy_top1(output, target),
                )
            )
            if args.dry_run:
                break

    epoch_loss = running_loss / max(1, seen)
    epoch_acc = 100.0 * (running_acc / max(1, seen))
    return epoch_loss, epoch_acc


@torch.no_grad()
def evaluate(model, device, loader, split_name="val"):
    model.eval()
    loss_sum = 0.0
    correct = 0
    total = 0

    for data, target in loader:
        data = data.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)

        output = model(data)
        loss = nn.functional.cross_entropy(output, target, reduction="sum")
        loss_sum += loss.item()

        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)

    avg_loss = loss_sum / max(1, total)
    acc = 100.0 * correct / max(1, total)

    print(
        "\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            split_name.capitalize(), avg_loss, correct, total, acc
        )
    )
    return avg_loss, acc


# ────────────────────────────────────────────────────────────────
# Model
# ────────────────────────────────────────────────────────────────
def build_model(num_classes: int):
    weights = MobileNet_V3_Small_Weights.DEFAULT
    model = mobilenet_v3_small(weights=weights)
    in_f = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(in_f, num_classes)
    return model, weights


# ────────────────────────────────────────────────────────────────
# Main
# ────────────────────────────────────────────────────────────────
def main():
    parser = argparse.ArgumentParser(
        description="Fruit/Veg Baseline (MobileNetV3-Small) — No Dendrites"
    )

    # ✅ Your Kaggle dataset structure (already split)
    parser.add_argument(
        "--train-dir",
        type=str,
        default="/kaggle/input/fruit-and-vegetable-image-recognition/train",
    )
    parser.add_argument(
        "--val-dir",
        type=str,
        default="/kaggle/input/fruit-and-vegetable-image-recognition/validation",
    )
    parser.add_argument(
        "--test-dir",
        type=str,
        default="/kaggle/input/fruit-and-vegetable-image-recognition/test",
    )

    # Training hyperparams
    parser.add_argument("--batch-size", type=int, default=64, metavar="N")
    parser.add_argument("--test-batch-size", type=int, default=128, metavar="N")
    parser.add_argument("--epochs", type=int, default=8, metavar="N")
    parser.add_argument("--lr", type=float, default=3e-4, metavar="LR")
    parser.add_argument("--gamma", type=float, default=0.9, metavar="M")
    parser.add_argument("--weight-decay", type=float, default=1e-4, metavar="WD")

    # Runtime
    parser.add_argument("--no-cuda", action="store_true", default=False)
    parser.add_argument("--dry-run", action="store_true", default=False)
    parser.add_argument("--seed", type=int, default=42, metavar="S")
    parser.add_argument("--log-interval", type=int, default=20, metavar="N")
    parser.add_argument("--num-workers", type=int, default=2)

    # Outputs
    parser.add_argument("--save-model", action="store_true", default=True)
    parser.add_argument("--out-dir", type=str, default="./outputs")
    parser.add_argument("--model-name", type=str, default="fruitveg_mnv3s_baseline.pt")
    parser.add_argument("--metrics-name", type=str, default="baseline_metrics.json")

    args = parser.parse_args()

    # Validate dataset dirs
    for p in [args.train_dir, args.val_dir, args.test_dir]:
        if not os.path.isdir(p):
            raise FileNotFoundError(f"Directory not found: {p}")

    use_cuda = (not args.no_cuda) and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print("[INFO] device:", device)

    set_seed(args.seed)
    ensure_dir(args.out_dir)

    # Build once to get weights object (for eval transforms)
    _, weights = build_model(num_classes=2)

    # ImageNet normalization (explicit + stable)
    IMAGENET_MEAN = (0.485, 0.456, 0.406)
    IMAGENET_STD = (0.229, 0.224, 0.225)

    # Train: light camera-style augmentation
    train_transform = transforms.Compose(
        [
            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(
                brightness=0.15, contrast=0.15, saturation=0.10
            ),
            transforms.ToTensor(),
            transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
        ]
    )

    # Val/Test: EXACT pretrained eval pipeline
    eval_transform = weights.transforms()

    train_ds = datasets.ImageFolder(args.train_dir, transform=train_transform)
    val_ds = datasets.ImageFolder(args.val_dir, transform=eval_transform)
    test_ds = datasets.ImageFolder(args.test_dir, transform=eval_transform)

    # Ensure class mapping consistency
    if val_ds.class_to_idx != train_ds.class_to_idx:
        raise ValueError(
            "Class mapping mismatch train vs validation.\n"
            f"train: {train_ds.class_to_idx}\nval: {val_ds.class_to_idx}"
        )
    if test_ds.class_to_idx != train_ds.class_to_idx:
        raise ValueError(
            "Class mapping mismatch train vs test.\n"
            f"train: {train_ds.class_to_idx}\ntest: {test_ds.class_to_idx}"
        )

    num_classes = len(train_ds.classes)
    print("[INFO] num_classes:", num_classes)
    print("[INFO] classes:", train_ds.classes)

    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=args.num_workers,
        pin_memory=use_cuda,
    )
    val_loader = torch.utils.data.DataLoader(
        val_ds,
        batch_size=args.test_batch_size,
        shuffle=False,
        num_workers=args.num_workers,
        pin_memory=use_cuda,
    )
    test_loader = torch.utils.data.DataLoader(
        test_ds,
        batch_size=args.test_batch_size,
        shuffle=False,
        num_workers=args.num-workers if False else args.num_workers,  # keep consistent
        pin_memory=use_cuda,
    )

    # Build model
    model, _ = build_model(num_classes=num_classes)
    model = model.to(device)

    params = count_params(model)
    print(f"[INFO] trainable params: {params:,} ({params/1e6:.2f}M)")

    optimizer = optim.AdamW(
        model.parameters(), lr=args.lr, weight_decay=args.weight_decay
    )
    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

    best_val_acc = -1.0
    best_state = None
    history = []

    start_time = time.time()

    for epoch in range(1, args.epochs + 1):
        tr_loss, tr_acc = train_one_epoch(
            args, model, device, train_loader, optimizer, epoch
        )
        val_loss, val_acc = evaluate(
            model, device, val_loader, split_name="validation"
        )

        scheduler.step()

        history.append(
            {
                "epoch": epoch,
                "train_loss": tr_loss,
                "train_acc": tr_acc,
                "val_loss": val_loss,
                "val_acc": val_acc,
                "lr": optimizer.param_groups[0]["lr"],
            }
        )

        # Save best
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = {
                "model_state": model.state_dict(),
                "classes": train_ds.classes,
                "class_to_idx": train_ds.class_to_idx,
                "best_val_acc": best_val_acc,
                "params": params,
                "args": vars(args),
            }
            if args.save_model:
                out_path = os.path.join(args.out_dir, args.model_name)
                torch.save(best_state, out_path)
                print(f"[INFO] Saved best model to {out_path}")

        if args.dry_run:
            break

    # Evaluate best checkpoint on test set
    if best_state is not None and args.save_model:
        ckpt_path = os.path.join(args.out_dir, args.model_name)
        ckpt = torch.load(ckpt_path, map_location=device)
        model.load_state_dict(ckpt["model_state"])

    test_loss, test_acc = evaluate(model, device, test_loader, split_name="test")

    elapsed = time.time() - start_time

    metrics = {
        "best_val_acc": best_val_acc,
        "final_test_acc": test_acc,
        "final_test_loss": test_loss,
        "params": params,
        "params_million": params / 1e6,
        "elapsed_seconds": elapsed,
        "history": history,
    }

    metrics_path = os.path.join(args.out_dir, args.metrics_name)
    with open(metrics_path, "w") as f:
        json.dump(metrics, f, indent=2)

    print(f"[DONE] Metrics saved to: {metrics_path}")
    print(f"[DONE] Best Val Acc: {best_val_acc:.2f}% | Test Acc: {test_acc:.2f}%")
    print(f"[DONE] Time: {elapsed/60:.1f} minutes")


if __name__ == "__main__":
    import sys

    sys.argv = [""]  # 👈 wipes notebook-injected args
    main()

[INFO] device: cuda
[INFO] num_classes: 36
[INFO] classes: ['apple', 'banana', 'beetroot', 'bell pepper', 'cabbage', 'capsicum', 'carrot', 'cauliflower', 'chilli pepper', 'corn', 'cucumber', 'eggplant', 'garlic', 'ginger', 'grapes', 'jalepeno', 'kiwi', 'lemon', 'lettuce', 'mango', 'onion', 'orange', 'paprika', 'pear', 'peas', 'pineapple', 'pomegranate', 'potato', 'raddish', 'soy beans', 'spinach', 'sweetcorn', 'sweetpotato', 'tomato', 'turnip', 'watermelon']
[INFO] trainable params: 1,554,756 (1.55M)
Train Epoch: 1 [0/3115 (0%)]	Loss: 3.693619	Acc: 1.56%


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Epoch: 1 [1280/3115 (41%)]	Loss: 2.610356	Acc: 45.31%
Train Epoch: 1 [2560/3115 (82%)]	Loss: 1.402538	Acc: 64.06%

Validation set: Average loss: 0.8386, Accuracy: 264/351 (75.21%)

[INFO] Saved best model to ./outputs/fruitveg_mnv3s_baseline.pt
Train Epoch: 2 [0/3115 (0%)]	Loss: 0.885908	Acc: 79.69%


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Epoch: 2 [1280/3115 (41%)]	Loss: 0.818186	Acc: 78.12%
Train Epoch: 2 [2560/3115 (82%)]	Loss: 0.670363	Acc: 85.94%

Validation set: Average loss: 0.4060, Accuracy: 304/351 (86.61%)

[INFO] Saved best model to ./outputs/fruitveg_mnv3s_baseline.pt
Train Epoch: 3 [0/3115 (0%)]	Loss: 0.429063	Acc: 87.50%


...

Train Epoch: 7 [1280/3115 (41%)]	Loss: 0.101401	Acc: 100.00%
Train Epoch: 7 [2560/3115 (82%)]	Loss: 0.148472	Acc: 95.31%

Validation set: Average loss: 0.1798, Accuracy: 333/351 (94.87%)

[INFO] Saved best model to ./outputs/fruitveg_mnv3s_baseline.pt
Train Epoch: 8 [0/3115 (0%)]	Loss: 0.073001	Acc: 98.44%


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Epoch: 8 [1280/3115 (41%)]	Loss: 0.108986	Acc: 96.88%


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Epoch: 8 [2560/3115 (82%)]	Loss: 0.180230	Acc: 95.31%

Validation set: Average loss: 0.1786, Accuracy: 334/351 (95.16%)

[INFO] Saved best model to ./outputs/fruitveg_mnv3s_baseline.pt

Test set: Average loss: 0.1760, Accuracy: 342/359 (95.26%)

[DONE] Metrics saved to: ./outputs/baseline_metrics.json
[DONE] Best Val Acc: 95.16% | Test Acc: 95.26%
[DONE] Time: 10.6 minutes


In [6]:
import json

# Define the path
file_path = './outputs/baseline_metrics.json'

# Load the data
with open(file_path, 'r') as f:
    metrics = json.load(f)

# Example: Print a specific metric
print(metrics)


{'best_val_acc': 95.15669515669515, 'final_test_acc': 95.26462395543176, 'final_test_loss': 0.1759721615188301, 'params': 1554756, 'params_million': 1.554756, 'elapsed_seconds': 635.7768702507019, 'history': [{'epoch': 1, 'train_loss': 2.3420290356845763, 'train_acc': 44.462279310961215, 'val_loss': 0.8386318717587028, 'val_acc': 75.21367521367522, 'lr': 0.00027}, {'epoch': 2, 'train_loss': 0.8013119672312974, 'train_acc': 77.30337076738213, 'val_loss': 0.40603443700024205, 'val_acc': 86.6096866096866, 'lr': 0.000243}, {'epoch': 3, 'train_loss': 0.47641743924797636, 'train_acc': 85.81059390622195, 'val_loss': 0.2879466738795962, 'val_acc': 89.45868945868946, 'lr': 0.0002187}, {'epoch': 4, 'train_loss': 0.34535526584469106, 'train_acc': 89.5666131678592, 'val_loss': 0.2369638318010205, 'val_acc': 91.16809116809117, 'lr': 0.00019683}, {'epoch': 5, 'train_loss': 0.26690456272128305, 'train_acc': 91.81380412743141, 'val_loss': 0.2088274290079405, 'val_acc': 93.44729344729345, 'lr': 0.00017